In [1]:
import awkward as ak
import hist
import numpy as np
from coffea import util as cutil
from coffea.lookup_tools.dense_lookup import dense_lookup
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory

btagWPs = {
    "deepJet": {
        "2016preVFP_UL": {
            "L": 0.0508,
            "M": 0.2598,
            "T": 0.6502,
        },
        "2016postVFP_UL": {
            "L": 0.0480,
            "M": 0.2489,
            "T": 0.6377,
        },
        "2017_UL": {
            "L": 0.0532,
            "M": 0.3040,
            "T": 0.7476,
        },
        "2018_UL": {
            "L": 0.0490,
            "M": 0.2783,
            "T": 0.7100,
        },
    },
    "deepCSV": {
        "2016preVFP_UL": {
            "L": 0.2027,
            "M": 0.6001,
            "T": 0.8819,
        },
        "2016postVFP_UL": {
            "L": 0.1918,
            "M": 0.5847,
            "T": 0.8767,
        },
        "2017_UL": {
            "L": 0.1355,
            "M": 0.4506,
            "T": 0.7738,
        },
        "2018_UL": {
            "L": 0.1208,
            "M": 0.4168,
            "T": 0.7665,
        },
    },
}

# single TT files to derive efficiency

taggerBranch = {"deepJet": "btagDeepFlavB", "deepCSV": "btagDeepB"}


file1 = "/uscms_data/d3/jieun201/YOURWORKINGAREA/nano_mc2018_2-15.root"

events = NanoEventsFactory.from_root(
    file1,
entry_stop=100_000, schemaclass=NanoAODSchema,
).events()
year = '2018_UL'

#events = file1.events()

# b-tagging only applied for jets with |eta| < 2.5
phasespace_cuts = (events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.5)
jets = ak.flatten(events.Jet[phasespace_cuts])

bins = [40,60,80,100,120,140,160,180,200,220,240,260,280,300,1000]


#for tag in ["deepJet", "deepCSV"]:
for tag in ["deepJet"]:
    #for wp in ["L", "M", "T"]:
    for wp in ["M"]:
        efficiencyinfo = (
            hist.Hist.new.Variable(bins, name="pt")
            .Reg(4, 0, 2.5, name="abseta")
            .IntCat([0, 4, 5], name="flavor")
            .Bool(name="passWP")
            .Double()
            .fill(
                pt=jets.pt,
                abseta=abs(jets.eta),
                flavor=jets.hadronFlavour,
                passWP=jets[taggerBranch[tag]] > btagWPs[tag][year][wp],
            )
        )
        eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
        efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
        #print(tag, wp, efflookup(np.array([42, 60]), 0.2, 2))

    #    cutil.save(efflookup, f"btageff_{tag}_{wp}_{year}.coffea")

/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1937 branches) at 0x7f1ba96421d0>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1937 branches) at 0x7f1ba96421d0>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1937 branches) at 0x7f1ba96421d0>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1937 branches) at 0x7f1ba96421d0>, taking first instance
  warnings.warn(
/home/jieun201/.loca

In [2]:
print(eff)

Hist(
  Variable([40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 1000], name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Double()) # Sum: 55.17669109358745 (nan with flow)


In [3]:
#s_2d = eff.project("pt","abseta")
#s_2d[{"flavor": 4j}]

In [4]:
eff_4=eff[{"flavor": 4j}]

In [5]:
eff_4.values()

array([[0.16682708, 0.15991157, 0.15822454, 0.15741583],
       [0.15064819, 0.14890198, 0.14739229, 0.15384615],
       [0.15260197, 0.16372392, 0.15932642, 0.14553991],
       [0.13463751, 0.12846715, 0.15351812, 0.15272727],
       [0.16992188, 0.14772727, 0.1827957 , 0.13793103],
       [0.1202346 , 0.1529052 , 0.12222222, 0.12941176],
       [0.15021459, 0.12025316, 0.1627907 , 0.1875    ],
       [0.18831169, 0.125     , 0.14457831, 0.13043478],
       [0.1954023 , 0.11688312, 0.2       , 0.08333333],
       [0.21126761, 0.12280702, 0.125     , 0.2       ],
       [0.15217391, 0.14285714, 0.23333333, 0.        ],
       [0.08333333, 0.08695652, 0.16666667, 0.25      ],
       [0.18518519, 0.14285714, 0.23076923, 0.        ],
       [0.18181818, 0.2875    , 0.26190476, 0.26666667]])

In [6]:
eff_5=eff[{"flavor": 5j}]

In [7]:
eff_5.values()

array([[0.83130289, 0.81678487, 0.78661654, 0.74211957],
       [0.85112048, 0.83864515, 0.78445719, 0.77493606],
       [0.86314469, 0.85078994, 0.8003992 , 0.77990169],
       [0.86227689, 0.84998463, 0.80175196, 0.74382434],
       [0.87926303, 0.85130478, 0.84128225, 0.77272727],
       [0.87427838, 0.85627376, 0.81521739, 0.80434783],
       [0.875     , 0.8515625 , 0.8280543 , 0.80368098],
       [0.87617555, 0.86307054, 0.81090909, 0.775     ],
       [0.85128205, 0.86557377, 0.75862069, 0.77419355],
       [0.87591241, 0.82901554, 0.85416667, 0.74358974],
       [0.83695652, 0.82432432, 0.86111111, 0.73684211],
       [0.82758621, 0.88172043, 0.79069767, 0.63636364],
       [0.82758621, 0.84285714, 0.82758621, 0.8       ],
       [0.84848485, 0.8241206 , 0.73786408, 0.63157895]])

In [8]:
eff_0=eff[{"flavor": 0j}]

In [9]:
eff_0.values()

array([[0.01542078, 0.01510731, 0.01787759, 0.0239293 ],
       [0.01004108, 0.00963983, 0.01332943, 0.01077111],
       [0.00971021, 0.01071811, 0.01254826, 0.01287375],
       [0.0084399 , 0.00969991, 0.01033912, 0.0131397 ],
       [0.00568182, 0.0075643 , 0.01166559, 0.01598174],
       [0.00765484, 0.00924499, 0.01371429, 0.00519031],
       [0.0105042 , 0.00367197, 0.01208981, 0.02910053],
       [0.00516351, 0.00924214, 0.00746269, 0.02347418],
       [0.00492611, 0.01709402, 0.00749064, 0.00641026],
       [0.01355932, 0.01149425, 0.00483092, 0.01904762],
       [0.0097561 , 0.02083333, 0.01612903, 0.06578947],
       [0.        , 0.00694444, 0.00934579, 0.01666667],
       [0.01626016, 0.00909091, 0.015625  , 0.        ],
       [0.01742919, 0.01877934, 0.04923077, 0.06206897]])